In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import time

from sklearn.linear_model.stochastic_gradient import SGDRegressor

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from math import sqrt

pd.options.display.max_columns = None

In [2]:
#Importing dataset
df = pd.read_csv("../processed/train_rand.csv", index_col=0)
df_val = pd.read_csv("../processed/val1_rand.csv", index_col=0)
df_val2 = pd.read_csv("../processed/val2.csv", index_col=0)

In [3]:
print(df.shape)
print(df_val.shape)
print(df_val2.shape)

(356953, 47)
(39622, 47)
(36911, 47)


In [4]:
df.head()

,StoreID,Date,IsHoliday,HasPromotions,StoreType,NearestCompetitor,Region,NumberOfSales,Region_AreaKM2,Region_GDP,Region_PopulationK,CloudCover,Max_Dew_PointC,Max_Gust_SpeedKm_h,Max_Humidity,Max_Sea_Level_PressurehPa,Max_TemperatureC,Max_VisibilityKm,Max_Wind_SpeedKm_h,Mean_Dew_PointC,Mean_Humidity,Mean_Sea_Level_PressurehPa,Mean_TemperatureC,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees,AssortmentType_With Fish Department,AssortmentType_With Non-Food Department,Rain,Snow,Fog,Hail,Thunderstorm,IsSaturday,IsSunday,OrdinalDate,WasOpenYesterday,IsOpenTomorrow,MeanMonthSales,MeanStoreSales,StdStoreSales
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1000,2016-03-01,0,0,2,326,7,5676.0,9643,17130,2770,8.0,1,35.72,100,1032,2,19.0,21,-1,82,1030,1,11.0,16,-2,70,1029,1,6.0,0.0,23,0,0,1,1,0,0,0,0,0,0,1,1,4769.820134,7675.446488,1890.559268
1,1000,2016-03-02,0,0,2,326,7,8111.0,9643,17130,2770,8.0,0,28.05,87,1030,5,23.0,16,-1,73,1027,3,13.0,10,-2,58,1025,1,10.0,0.0,56,0,0,0,1,0,0,0,0,0,1,1,0,4769.820134,7675.446488,1890.559268
2,1000,2016-03-04,0,0,2,326,7,8300.0,9643,17130,2770,8.0,0,34.94,81,1026,4,31.0,23,-1,71,1024,3,11.0,10,-3,55,1023,2,8.0,0.0,22,0,0,1,0,0,0,0,0,0,3,1,1,4769.820134,7675.446488,1890.559268
3,1000,2016-03-05,0,0,2,326,7,7154.0,9643,17130,2770,6.0,-3,33.75,80,1027,8,31.0,19,-4,56,1024,3,15.0,10,-6,25,1022,-1,10.0,0.0,108,0,0,0,0,0,0,0,1,0,4,1,0,4769.820134,7675.446488,1890.559268
5,1000,2016-03-07,0,1,2,326,7,10110.0,9643,17130,2770,8.0,-2,34.37,93,1024,11,31.0,24,-3,65,1020,4,22.0,11,-4,26,1016,-2,16.0,0.0,68,0,0,0,0,0,0,0,0,0,6,1,1,4769.820134,7675.446488,1890.559268


We save 'Date', 'StoreID' and 'Region' for the final csv with all the results.

In [5]:
date_val2 = df_val2['Date']
store_val2 = df_val2['StoreID']
region_val2 = df_val2['Region']

We drop them because they're useless for the computation of the model.

In [6]:
df = df.drop(['Date', 'StoreID', 'Region'], axis=1)
df_val = df_val.drop(['Date', 'StoreID', 'Region'], axis=1)
df_val2 = df_val2.drop(['Date', 'StoreID', 'Region'], axis=1)

In [7]:
y = df['NumberOfSales']
y_val = df_val['NumberOfSales']
y_val2 = df_val2['NumberOfSales']

df = df.drop(['NumberOfSales'], axis=1)
df_val = df_val.drop(['NumberOfSales'], axis=1)
df_val2 = df_val2.drop(['NumberOfSales'], axis=1)

We normalize the values and set the 'OrdinalDate' on the day of of the year

In [8]:
df.OrdinalDate = df.OrdinalDate.map(lambda x: x%365)
df_val.OrdinalDate = df_val.OrdinalDate.map(lambda x: x%365)
df_val2.OrdinalDate = df_val2.OrdinalDate.map(lambda x: x%365)

In [9]:
X = (df - df.min()) / (df.max() - df.min())
X_val = (df_val - df_val.min()) / (df_val.max() - df_val.min())
X_val2 = (df_val2 - df_val2.min()) / (df_val2.max() - df_val2.min())

In [10]:
X.head()

,IsHoliday,HasPromotions,StoreType,NearestCompetitor,Region_AreaKM2,Region_GDP,Region_PopulationK,CloudCover,Max_Dew_PointC,Max_Gust_SpeedKm_h,Max_Humidity,Max_Sea_Level_PressurehPa,Max_TemperatureC,Max_VisibilityKm,Max_Wind_SpeedKm_h,Mean_Dew_PointC,Mean_Humidity,Mean_Sea_Level_PressurehPa,Mean_TemperatureC,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees,AssortmentType_With Fish Department,AssortmentType_With Non-Food Department,Rain,Snow,Fog,Hail,Thunderstorm,IsSaturday,IsSunday,OrdinalDate,WasOpenYesterday,IsOpenTomorrow,MeanMonthSales,MeanStoreSales,StdStoreSales
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.666667,0.003281,0.291715,0.515529,0.266576,1.00,0.405405,0.173176,1.000000,0.872340,0.270833,0.612903,0.183673,0.400000,0.742857,0.901961,0.325581,0.354839,0.341463,0.444444,0.666667,0.892857,0.394737,0.193548,0.0,0.066482,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,1.0,0.128502,0.430061,0.482451
1,0.0,0.0,0.666667,0.003281,0.291715,0.515529,0.266576,1.00,0.378378,0.082941,0.767857,0.829787,0.333333,0.741935,0.132653,0.400000,0.614286,0.843137,0.372093,0.419355,0.195122,0.444444,0.533333,0.821429,0.394737,0.322581,0.0,0.157895,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.002747,1.0,0.0,0.128502,0.430061,0.482451
2,0.0,0.0,0.666667,0.003281,0.291715,0.515529,0.266576,1.00,0.378378,0.164000,0.660714,0.744681,0.312500,1.000000,0.204082,0.400000,0.585714,0.784314,0.372093,0.354839,0.195122,0.416667,0.500000,0.785714,0.421053,0.258065,0.0,0.063712,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008242,1.0,1.0,0.128502,0.430061,0.482451
3,0.0,0.0,0.666667,0.003281,0.291715,0.515529,0.266576,0.75,0.297297,0.150000,0.642857,0.765957,0.395833,1.000000,0.163265,0.314286,0.371429,0.784314,0.372093,0.483871,0.195122,0.333333,0.166667,0.767857,0.342105,0.322581,0.0,0.301939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.010989,1.0,0.0,0.128502,0.430061,0.482451
5,0.0,1.0,0.666667,0.003281,0.291715,0.515529,0.266576,1.00,0.324324,0.157294,0.875000,0.702128,0.458333,1.000000,0.214286,0.342857,0.500000,0.705882,0.395349,0.709677,0.219512,0.388889,0.177778,0.660714,0.315789,0.516129,0.0,0.191136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016484,1.0,1.0,0.128502,0.430061,0.482451


We try to find the best parameters

In [11]:
dict_params = {'loss_function' : [], 'alpha' : [], 'epsilon' : [], 'learning_rate' : [], 'mse' : [], 'mae' : []}

In [12]:
loss_fun = ['squared_loss', 'epsilon_insensitive']
learning_r = ['constant', 'optimal', 'invscaling']

start = time.time()

for l_f in loss_fun:
    for l_r in learning_r:
        for a in np.linspace(.000001, .00001 , 100):
            for e in np.linspace(.1, .01, 5):
                sgd = SGDRegressor(loss=l_f, alpha=a, epsilon=e, learning_rate=l_r, shuffle=True, random_state=1234567890)
                dict_params['loss_function'].append(l_f)
                dict_params['alpha'].append(a)
                dict_params['epsilon'].append(e)
                dict_params['learning_rate'].append(l_r)
                sgd.fit(X, y)
                y_pred = sgd.predict(X_val)
                mse = sqrt(mean_squared_error(y_val,y_pred))
                mae = mean_absolute_error(y_val, y_pred)
                dict_params['mse'].append(mse)
                dict_params['mae'].append(mae)
                
                print(l_f, l_r, a, e, mse, mae)
            
end = time.time() - start
end

squared_loss constant 1e-06 0.1 997.5107515214556 728.1210691295976
squared_loss constant 1e-06 0.0775 997.5107515214556 728.1210691295976
squared_loss constant 1e-06 0.055 997.5107515214556 728.1210691295976
squared_loss constant 1e-06 0.0325 997.5107515214556 728.1210691295976
squared_loss constant 1e-06 0.01 997.5107515214556 728.1210691295976
squared_loss constant 1.090909090909091e-06 0.1 997.5106843537545 728.1204416879145
squared_loss constant 1.090909090909091e-06 0.0775 997.5106843537545 728.1204416879145
squared_loss constant 1.090909090909091e-06 0.055 997.5106843537545 728.1204416879145
squared_loss constant 1.090909090909091e-06 0.0325 997.5106843537545 728.1204416879145
squared_loss constant 1.090909090909091e-06 0.01 997.5106843537545 728.1204416879145
squared_loss constant 1.1818181818181818e-06 0.1 997.510617686948 728.1198144128663
squared_loss constant 1.1818181818181818e-06 0.0775 997.510617686948 728.1198144128663
squared_loss constant 1.1818181818181818e-06 0.055 

squared_loss constant 2.7272727272727276e-06 0.0775 997.5095604768753 728.1092435593777
squared_loss constant 2.7272727272727276e-06 0.055 997.5095604768753 728.1092435593777
squared_loss constant 2.7272727272727276e-06 0.0325 997.5095604768753 728.1092435593777
squared_loss constant 2.7272727272727276e-06 0.01 997.5095604768753 728.1092435593777
squared_loss constant 2.8181818181818185e-06 0.1 997.5095027363249 728.1086276467066
squared_loss constant 2.8181818181818185e-06 0.0775 997.5095027363249 728.1086276467066
squared_loss constant 2.8181818181818185e-06 0.055 997.5095027363249 728.1086276467066
squared_loss constant 2.8181818181818185e-06 0.0325 997.5095027363249 728.1086276467066
squared_loss constant 2.8181818181818185e-06 0.01 997.5095027363249 728.1086276467066
squared_loss constant 2.9090909090909093e-06 0.1 997.509445486757 728.1080136524345
squared_loss constant 2.9090909090909093e-06 0.0775 997.509445486757 728.1080136524345
squared_loss constant 2.9090909090909093e-06 0

squared_loss constant 4.454545454545455e-06 0.01 997.5085468725877 728.097663604464
squared_loss constant 4.5454545454545455e-06 0.1 997.5084983741741 728.0970597793781
squared_loss constant 4.5454545454545455e-06 0.0775 997.5084983741741 728.0970597793781
squared_loss constant 4.5454545454545455e-06 0.055 997.5084983741741 728.0970597793781
squared_loss constant 4.5454545454545455e-06 0.0325 997.5084983741741 728.0970597793781
squared_loss constant 4.5454545454545455e-06 0.01 997.5084983741741 728.0970597793781
squared_loss constant 4.636363636363636e-06 0.1 997.508450357188 728.0964561069628
squared_loss constant 4.636363636363636e-06 0.0775 997.508450357188 728.0964561069628
squared_loss constant 4.636363636363636e-06 0.055 997.508450357188 728.0964561069628
squared_loss constant 4.636363636363636e-06 0.0325 997.508450357188 728.0964561069628
squared_loss constant 4.636363636363636e-06 0.01 997.508450357188 728.0964561069628
squared_loss constant 4.727272727272727e-06 0.1 997.508402

squared_loss constant 6.272727272727273e-06 0.055 997.5076678173132 728.0856768853492
squared_loss constant 6.272727272727273e-06 0.0325 997.5076678173132 728.0856768853492
squared_loss constant 6.272727272727273e-06 0.01 997.5076678173132 728.0856768853492
squared_loss constant 6.363636363636363e-06 0.1 997.5076288547152 728.0850826528902
squared_loss constant 6.363636363636363e-06 0.0775 997.5076288547152 728.0850826528902
squared_loss constant 6.363636363636363e-06 0.055 997.5076288547152 728.0850826528902
squared_loss constant 6.363636363636363e-06 0.0325 997.5076288547152 728.0850826528902
squared_loss constant 6.363636363636363e-06 0.01 997.5076288547152 728.0850826528902
squared_loss constant 6.454545454545455e-06 0.1 997.5075903638473 728.0844885688144
squared_loss constant 6.454545454545455e-06 0.0775 997.5075903638473 728.0844885688144
squared_loss constant 6.454545454545455e-06 0.055 997.5075903638473 728.0844885688144
squared_loss constant 6.454545454545455e-06 0.0325 997.5

squared_loss constant 8.000000000000001e-06 0.01 997.5070077364597 728.0744775600309
squared_loss constant 8.090909090909092e-06 0.1 997.5069776562791 728.0738923503159
squared_loss constant 8.090909090909092e-06 0.0775 997.5069776562791 728.0738923503159
squared_loss constant 8.090909090909092e-06 0.055 997.5069776562791 728.0738923503159
squared_loss constant 8.090909090909092e-06 0.0325 997.5069776562791 728.0738923503159
squared_loss constant 8.090909090909092e-06 0.01 997.5069776562791 728.0738923503159
squared_loss constant 8.181818181818183e-06 0.1 997.506948038945 728.0733082508277
squared_loss constant 8.181818181818183e-06 0.0775 997.506948038945 728.0733082508277
squared_loss constant 8.181818181818183e-06 0.055 997.506948038945 728.0733082508277
squared_loss constant 8.181818181818183e-06 0.0325 997.506948038945 728.0733082508277
squared_loss constant 8.181818181818183e-06 0.01 997.506948038945 728.0733082508277
squared_loss constant 8.272727272727274e-06 0.1 997.5069188839

...

epsilon_insensitive invscaling 4.454545454545455e-06 0.0325 2316.2890723252895 1599.9401913782224
epsilon_insensitive invscaling 4.454545454545455e-06 0.01 2316.2889581559853 1599.9401585102821
epsilon_insensitive invscaling 4.5454545454545455e-06 0.1 2316.308958848481 1599.9490159616894
epsilon_insensitive invscaling 4.5454545454545455e-06 0.0775 2316.305390576518 1599.9475712416825
epsilon_insensitive invscaling 4.5454545454545455e-06 0.055 2316.3058043129404 1599.947715215944
epsilon_insensitive invscaling 4.5454545454545455e-06 0.0325 2316.3069205139145 1599.9481468810898
epsilon_insensitive invscaling 4.5454545454545455e-06 0.01 2316.30845054448 1599.948791379587
epsilon_insensitive invscaling 4.636363636363636e-06 0.1 2316.327565545327 1599.957310214496
epsilon_insensitive invscaling 4.636363636363636e-06 0.0775 2316.3271960805478 1599.9571848326887
epsilon_insensitive invscaling 4.636363636363636e-06 0.055 2316.3268077845305 1599.9570386335247
epsilon_insensitive invscaling 4.63

epsilon_insensitive invscaling 6.090909090909091e-06 0.1 2316.630238351875 1600.0918056271219
epsilon_insensitive invscaling 6.090909090909091e-06 0.0775 2316.628991556081 1600.0913206746495
epsilon_insensitive invscaling 6.090909090909091e-06 0.055 2316.626445448938 1600.0902276612808
epsilon_insensitive invscaling 6.090909090909091e-06 0.0325 2316.6271472567705 1600.0904182739332
epsilon_insensitive invscaling 6.090909090909091e-06 0.01 2316.627346179712 1600.0905653172358
epsilon_insensitive invscaling 6.181818181818182e-06 0.1 2316.6464241375156 1600.0991210189663
epsilon_insensitive invscaling 6.181818181818182e-06 0.0775 2316.647102880488 1600.0993814443964
epsilon_insensitive invscaling 6.181818181818182e-06 0.055 2316.647229279119 1600.0994407361666
epsilon_insensitive invscaling 6.181818181818182e-06 0.0325 2316.648759503343 1600.100137376012
epsilon_insensitive invscaling 6.181818181818182e-06 0.01 2316.64874759759 1600.1001496248584
epsilon_insensitive invscaling 6.272727272

epsilon_insensitive invscaling 7.636363636363636e-06 0.055 2316.938091221385 1600.2296105828336
epsilon_insensitive invscaling 7.636363636363636e-06 0.0325 2316.9372086889125 1600.2292721986391
epsilon_insensitive invscaling 7.636363636363636e-06 0.01 2316.9365258328953 1600.2289434776612
epsilon_insensitive invscaling 7.727272727272727e-06 0.1 2316.954791560911 1600.237139807202
epsilon_insensitive invscaling 7.727272727272727e-06 0.0775 2316.955388589069 1600.2374295785007
epsilon_insensitive invscaling 7.727272727272727e-06 0.055 2316.955716458731 1600.237589299849
epsilon_insensitive invscaling 7.727272727272727e-06 0.0325 2316.954615699845 1600.2369684899552
epsilon_insensitive invscaling 7.727272727272727e-06 0.01 2316.954058039679 1600.2367777980387
epsilon_insensitive invscaling 7.818181818181818e-06 0.1 2316.9723763960483 1600.2449481802564
epsilon_insensitive invscaling 7.818181818181818e-06 0.0775 2316.970855404338 1600.2443426412906
epsilon_insensitive invscaling 7.81818181

epsilon_insensitive invscaling 9.181818181818184e-06 0.0325 2317.243864920045 1600.366485848121
epsilon_insensitive invscaling 9.181818181818184e-06 0.01 2317.243258806442 1600.3662375019435
epsilon_insensitive invscaling 9.272727272727274e-06 0.1 2317.26021998564 1600.3737453167337
epsilon_insensitive invscaling 9.272727272727274e-06 0.0775 2317.263556062339 1600.375288839886
epsilon_insensitive invscaling 9.272727272727274e-06 0.055 2317.260933542876 1600.3741640348412
epsilon_insensitive invscaling 9.272727272727274e-06 0.0325 2317.2581646968106 1600.3729628822794
epsilon_insensitive invscaling 9.272727272727274e-06 0.01 2317.2593609048195 1600.373496726012
epsilon_insensitive invscaling 9.363636363636365e-06 0.1 2317.2754811647224 1600.3806565768796
epsilon_insensitive invscaling 9.363636363636365e-06 0.0775 2317.2749192441597 1600.3804009757334
epsilon_insensitive invscaling 9.363636363636365e-06 0.055 2317.2768013536806 1600.3812114405803
epsilon_insensitive invscaling 9.36363636

1957.4610810279846

In [13]:
params = pd.DataFrame(dict_params)

We extrapolate the best parameters

In [20]:
min_mae = params.mae.min()
best_params = params[(params['mae'] == min_mae)].iloc[0]

print("Minimum mae: %f" %min_mae)

loss_f = best_params['loss_function']
alp = best_params['alpha']
eps = best_params['epsilon']
learn_rate = best_params['learning_rate']

print("Loss function: " + loss_f + "; alpha: %f: epsilon: %f" %(alp, eps) + "; learning rate: " + learn_rate)

Minimum mae: 711.153174
Loss function: epsilon_insensitive; alpha: 0.000001: epsilon: 0.032500; learning rate: optimal


We validate our model on the last two months of the dataset

In [15]:
sgd = SGDRegressor(loss=loss_f, alpha=alp, epsilon=eps, learning_rate=learn_rate, shuffle=True, random_state=1234567890)
start = time.time()
sgd.fit(X, y)
y_pred2 = sgd.predict(X_val2)
end = time.time() - start
mse2 = sqrt(mean_squared_error(y_val2, y_pred2))
mae2 = mean_absolute_error(y_val2, y_pred2)

print("mse: %f, mae: %f, time: %f" %(mse2, mae2, end))

mse: 1251.222831, mae: 978.659063, time: 0.643600


In [16]:
dict_val2 = {'Date' : date_val2, 'StoreID' : store_val2, 'RegionID' : region_val2, 'SalesPredicted': [], 'SalesReal': []}

In [17]:
dict_val2['SalesReal'] = list(y_val2.to_frame()['NumberOfSales'].values)
dict_val2['SalesPredicted'] = list(y_pred2)

In [22]:
res = pd.DataFrame(dict_val2)

In [24]:
res.head(50)

,Date,StoreID,RegionID,SalesPredicted,SalesReal
index,,,,,
670,2018-01-01,1000,7,8367.068309,8540.0
671,2018-01-02,1000,7,7545.349766,10364.0
672,2018-01-03,1000,7,8155.534175,4676.0
674,2018-01-05,1000,7,8310.635742,6267.0
675,2018-01-06,1000,7,7315.520818,5953.0
677,2018-01-08,1000,7,9979.362176,9075.0
678,2018-01-09,1000,7,9324.756444,7856.0
679,2018-01-10,1000,7,9180.057377,7423.0
680,2018-01-11,1000,7,8853.194227,6283.0


In [19]:
res.to_csv("../results/result_linear_regression.csv")